In [ ]:
# @title Setup
from google.colab import auth
from google.cloud import bigquery
from google.colab import data_table
import networkx as nx
import pandas as pd

project = 'asymmetric-cove-187011' # Project ID inserted based on the query results selected to explore
location = 'US' # Location inserted based on the query results selected to explore
client = bigquery.Client(project=project, location=location)
data_table.enable_dataframe_formatter()
auth.authenticate_user()

MessageError: Error: credential propagation was unsuccessful

# Links Data


In [ ]:
links = client.get_job('bquxjob_1ed98894_18f3aba0e77')
links_df = links.to_dataframe()

In [ ]:
print(links.query)

SELECT *
FROM
  `glossy-odyssey-366820.farcaster.links`;


In [ ]:
links_df.head()

,fid,timestamp,target_fid,type,deleted_at
0,252365,2024-02-03 02:02:43+00:00,256209,follow,NaT
1,319720,2024-02-18 17:01:57+00:00,323282,follow,NaT
2,265276,2024-02-11 19:28:48+00:00,330196,follow,NaT
3,316960,2024-02-07 13:50:47+00:00,1237,follow,NaT
4,323008,2024-02-21 08:13:16+00:00,338645,follow,NaT


# Profile Data

In [ ]:
profiles = client.get_job('bquxjob_59d5669f_18f3af35231')
profiles_df = profiles.to_dataframe()

In [ ]:
print(profiles.query)

SELECT *
FROM
  `glossy-odyssey-366820.farcaster.profiles`;


In [ ]:
profiles_df.head()

,fid,last_updated_at,data,custody_address
0,504692,2024-04-28 08:29:33+00:00,None,0x77b0352100ebdb86a45557296e05721e48cc6d01
1,511997,2024-05-02 16:45:35+00:00,None,0xed414b0b2d8ba9208c26fd264177f0afd7ab1c7f
2,506795,2024-04-30 04:30:49+00:00,None,0xf3cb1a6ccb27f19ed26a95e1e5cd3f6c0cf802d4
3,502808,2024-04-26 18:05:15+00:00,None,0x9f839c3f59d73582fa26197cf69ab7bc92a7120f
4,510407,2024-05-01 22:34:01+00:00,None,0xfabb0fa8d23643e7aa1e6fb0f13b0128e9c238d1


# Convert ~1/5 of the tabular data to graph data (Proof of concept!)

In [ ]:
G = nx.from_pandas_edgelist(
    results[0:100000],
    source='fid',
    target='target_fid',
    create_using=nx.DiGraph()
)

# Run Pagerank on the fids




In [ ]:
pr = nx.pagerank(G)

# Process results to find top 10 fids by pagerank

In [ ]:
df_pr = pd.DataFrame.from_dict(pr,  orient='index', columns = ['pagerank']).reset_index(names=['fid'])

In [ ]:
top_10 = df_pr.sort_values(by='pagerank', ascending=False)[0:10]

In [ ]:
top10_profiles_df = pd.merge(profiles_df, top_10, on = 'fid').sort_values('pagerank', ascending=False)

In [ ]:
top10_profiles_df[['pagerank', 'fid', 'data']]

,pagerank,fid,data
3,0.004042,99,"{""bio"":""@base contributor #001; onchain cities..."
6,0.002239,206,"{""bio"":""Onchain is the new online - FID 206 - ..."
4,0.002120,11188,"{""bio"":""Blog: https://polynya.mirror.xyz/"",""di..."
1,0.002044,207,"{""bio"":""(summer of) protocol {support | guild ..."
2,0.002021,239,"{""bio"":""/ted | exploring the onchain frontier ..."
5,0.001966,129,"{""bio"":""Building /bright-moments - minting onc..."
0,0.001947,418777,"{""bio"":""Animator & Artist\nlinktr.ee/andrewbel..."
9,0.001801,457,"{""bio"":""code @ spindl.xyz. Interested in techn..."
8,0.001793,248172,"{""bio"":""artist. between galaxies. 🌈🌴🪐 linktr.e..."
7,0.001781,231317,"{""bio"":""SocialFi | Hypemaster @jam"",""display"":..."
